<a href="https://colab.research.google.com/github/elinoyam/FindMyParking/blob/WorkingWithColab/CalculateProbabilitiesFromGeojson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests

In [ ]:
def find_osm_way_id(coordinate):
  lon = coordinate[0]
  lat = coordinate[1]
  # get the way id - request json format of the reverse method of OSM librery nominatim
  request_url = "https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat=" + f'{lat:.4f}' + "&lon=" + f'{lon:.4f}'
  response = requests.get(request_url)
  way_id_response = json.loads(response.text)
  way_id = way_id_response ['osm_id'] 
  return way_id

In [ ]:
def calculate_cords(list_of_old_coorinates):
  transformed_coordinates = []
  for old_cord in list_of_old_coorinates:
    old_lat = old_cord[0]
    old_lon = old_cord[1]
    # get the latitude and longitude in the right coordinates 
    # from the original format EPSG:25833 to format: EPSG:4326
    request_url = "https://epsg.io/trans?s_srs=25833&t_srs=4326&x="+old_lat+"&y=" + old_lon
    response = requests.get(request_url)
    transform_coordinates_response = json.loads(response.text)
    trans_lon = transform_coordinates_response['x']
    trans_lat = transform_coordinates_response['y']
    transformed_coordinates.append([trans_lon, trans_lat])


In [ ]:
def transform_line(line):
  old_coords = line['coordinates'][0][0]
  transformed_coords = calculate_cords(old_coords)
  osm_id = find_osm_way_id(transformed_coords[0])
  # calculate parking area
  # calculate number of parking spots in the parking area
  # get the label (parking empty/occupied)
  